In [ ]:
from avcv.all import *
import torch, torch.nn as nn
from dms_drowsiness.video_writer import Board
import onnxruntime
import numpy as np, cv2
from PIL import Image
from ple.all import *

# Data

In [ ]:
VIDEO_RAW_DIR_FOOD = '/data/DMS_Behavior_Detection/RawVideos/Action_Eating/'
VIDEO_RAW_DIR_CIGARRET = '/data/DMS_Behavior_Detection/mobile_cigarret_foreignerUS'
VAL_USERS = ['hungng', 'chungtd12', 'thomp4', 'thuyhv5']
PRED_DIR_COCO ='.cache/raw_video_predict_face_food'

In [ ]:
def get_meta_jsons(input):
    """
        Asumming 
    """
    if input.endswith('.mp4'):
        video_path = input
        label_path = input.replace('.mp4', '.json')
    else:
        label_path = input
        video_path = input.replace('.json', '.mp4')

    video_name = get_name(label_path)
    
    pred_json_path = f'{PRED_DIR_COCO}/{video_name}/annotations/pred_mb2_face_food.json'

    assert osp.exists(pred_json_path), f'404 {pred_json_path}'
    assert osp.exists(video_path), f'404 {video_path}'
        
    if not osp.exists(label_path):
        logger.warning(f'404 {label_path=}')
        label_path = None
    
    return dict(
        pred_json_path=pred_json_path,
        label_path = label_path,
        video_path=video_path,
    )


list_json_paths = []
tobe_extract_videos = []

label_paths  = glob(f'{VIDEO_RAW_DIR_FOOD}/**/*.json', recursive=True)
for label_path in label_paths:
    try:
        json_paths = get_meta_jsons(label_path)
        list_json_paths.append(json_paths)
    except Exception as e:
        print(e)
        tobe_extract_videos.append(label_path.replace('.json', '.mp4'))
        pass

label_paths = glob(f'{VIDEO_RAW_DIR_CIGARRET}/*/**/*.json', recursive=True)

for label_path in label_paths:
    try:
        json_paths = get_meta_jsons(label_path)
        list_json_paths.append(json_paths)
    except Exception as e:
        print(e)
        tobe_extract_videos.append(label_path.replace('.json', '.mp4'))
        pass
len(list_json_paths)

109

In [ ]:
%%time
CACHE_DF_PATH = '/tmp/eating_cache_data_df.pkl'
if not osp.exists(CACHE_DF_PATH):
    list_json_paths = load_meta_jsons_with_label()
    
    print('Collecting data from scratch')
    data = multi_thread(collect_data, list(enumerate(list_json_paths)))
    all_data = []
    for _ in data: all_data += _

    df = pd.DataFrame(all_data, columns=['tensor', 'action', 'video_index', 'img_path', 'img_id', 'feat1d'])
    ids = df[df['action'] == 'smocking'].index
    df.loc[ids, 'action'] = 'smoking'

    df['x'] = df.feat1d.apply(convert_f)
    df['y'] = df['action'].apply(get_y)
else:
    print(f'Load data from {CACHE_DF_PATH=}')
    df = mmcv.load(CACHE_DF_PATH)

Load data from CACHE_DF_PATH='/tmp/eating_cache_data_df.pkl'
CPU times: user 66.9 ms, sys: 23.5 ms, total: 90.4 ms
Wall time: 88.7 ms


In [ ]:
# from scripts.export_onnx import model_wraper, mb2_yolox

In [ ]:
INP_SHAPE = (224, 416)

In [ ]:
class ImageDataset:
    @staticmethod
    def load_image_tensor(path):
        img = mmcv.imread(path, 0)
        img = mmcv.imrescale(img, INP_SHAPE)
        img = mmcv.impad(img, shape=INP_SHAPE, pad_val=114)
        img = img[None].astype(np.float32)
        return torch.from_numpy(img)


    def __init__(self, img_paths, y=None):
        self.img_paths = img_paths
        self.y = y if y is not None else [0]*len(img_paths)
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, i):
        return load_image_tensor(self.img_paths[i]), self.y[i]
    
ds = ImageDataset(df.img_path.values)

## Update detection to dataframe

In [ ]:
OBJECTS = ['face', 'cigarette', 'food/drink', 'phone']

imgpath2index = {v:k for k, v in df.img_path.to_dict().items()}

def update_one_image(img_id, cc):
    def get_max_score_ann(anns, catname):
        cat_id = catname2id[catname]
        anns_cat = [ann for ann in anns if ann['category_id'] == cat_id]
        anns_cat = list(sorted(anns_cat, key=lambda x:x['score']))
        if len(anns_cat):
            return anns_cat[-1]
        return None

    img = cc.gt.imgs[img_id]
    df['img_info'] = str(img)
    catname2id = {cat['name']:cat['id'] for cat in cc.gt.cats.values()}
    img_path = osp.abspath(osp.join(cc.img_dir, img['file_name']))
    anns = cc.gt.imgToAnns[img_id]
    index = imgpath2index[img_path]
    for class_name in OBJECTS:
        df.loc[index, class_name] = str(anns)
    return df

for json_paths in tqdm(list_json_paths):
    cc = CocoDataset(json_paths['pred_json_path'])
    for img_id in cc.img_ids:
        update_one_image(img_id, cc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 109/109 [01:45<00:00,  1.03it/s]


In [ ]:
_df = df[df.video_index==108]

In [ ]:
from yolox.tracker.byte_tracker import BYTETracker
from yolox.utils.visualize import plot_tracking
from tools.coco_tracking import make_parser


In [ ]:
for _, row in _df.iterrows():break

Namespace(aspect_ratio_thresh=10, categories=None, csv_out_path=None, match_thresh=1.1, min_box_area=10, output_dir='./results_vis', path='/home/anhvth8/.local/share/jupyter/runtime/kernel-b1d71de9-4593-421c-9970-e519b9e2b9d3.json', track_buffer=3000, track_thresh=0.1, verbose=True, visualize=False)

In [ ]:
# row

In [ ]:
# get_bboxes??

In [ ]:
# get_bboxes??

In [ ]:
cc = CocoDataset(list_json_paths[_df.iloc[0].video_index]['pred_json_path'])

In [ ]:
args = make_parser().parse_known_args()[0]
args.mot20 = False
tracker = dict()
for cat_name in OBJECTS:
    
    tracker[cat_name] = BYTETracker(args)

In [ ]:
cat_name2id = {cat['name']:cat['id'] for cat in cc.gt.cats.values()}

In [ ]:
results = [f"img_id,tid,cat_id,x,y,w,h,score\n"]
vis_imgs = []
img_ids = list(cc.gt.imgs.keys())
imsize = None
args.visualize=True
tracklets = dict()
pbar = tqdm(sorted(img_ids))

img_info = cc.gt.imgs[img_ids[0]]
imsize = (img_info['height'], img_info['width'])
for img_id in pbar:
    frame_id = img_id
    ret_val = True
    anns = cc.gt.imgToAnns[img_id]
    if args.visualize:
        online_im = mmcv.imresize(cc.imread(img_id), imsize[::-1])

    for class_name in OBJECTS:
        category_id = cat_name2id[class_name]
        outputs = torch.from_numpy(
            get_bboxes(anns,
                       mode='xyxy',
                       category_ids=[category_id],
                       score_thr=0.01, with_score=1))
        # print(len(outputs))
        if outputs is not None and len(outputs) > 0:
            online_targets = tracker[class_name].update(
                outputs, imsize, imsize)

            online_tlwhs = []
            online_ids = []
            online_scores = []
            for t in online_targets:
                tlwh = t.tlwh
                tid = t.track_id
                vertical = tlwh[2] / tlwh[3] > args.aspect_ratio_thresh
                if tlwh[2] * tlwh[3] > args.min_box_area and not vertical:
                    online_tlwhs.append(tlwh)
                    online_ids.append(tid)
                    online_scores.append(t.score)
                    results.append(
                        f"{frame_id},{tid},{category_id},{tlwh[0]:.2f},{tlwh[1]:.2f},{tlwh[2]:.2f},{tlwh[3]:.2f},{t.score:.2f}\n"
                    )

            if args.visualize:
                online_im = plot_tracking(online_im,
                                          online_tlwhs,
                                          online_ids,
                                          frame_id=frame_id + 1,
                                          fps=-1)
    if args.visualize:
        vis_imgs.append(online_im)
    frame_id += 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1069/1069 [00:05<00:00, 194.14it/s]


In [ ]:
images_to_video(vis_imgs, 'vis.mp4', output_size=(416, 224))

2022-09-29 09:57:17.808 | INFO     | avcv.utils:images_to_video:267 - Write video, output_size: (416, 224)


[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1069/1069, 478.5 task/s, elapsed: 2s, ETA:     0s

2022-09-29 09:57:20.045 | INFO     | avcv.utils:images_to_video:277 - -> /home/anhvth8/gitprojects/YOLOX/vis.mp4
